In [1]:
!conda install -y psycopg2

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 25.5.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [2]:
from sqlalchemy import create_engine
import pandas as pd

conn_str = (
    "postgresql+psycopg2://dbadmin:BZ6uHRGxki6a7qD"
    "@dcpostgres.postgres.database.azure.com:5432/DataChallenge"
)



In [3]:
import pandas as pd
import ast
import psycopg2

conn = psycopg2.connect(
    dbname="DataChallenge",
    user="dbadmin",
    password="BZ6uHRGxki6a7qD",
    host="dcpostgres.postgres.database.azure.com",
    port="5432",
    sslmode="require"
)

query = """
SELECT
  created_at,
  senti_raw_tabularis
FROM tweets
WHERE senti_raw_tabularis IS NOT NULL
  AND created_at      IS NOT NULL;
"""

df = pd.read_sql_query(query, conn)

df['label'] = df['senti_raw_tabularis'].apply(lambda x: ast.literal_eval(x)['label'])

df['created_at'] = pd.to_datetime(df['created_at'])


In [4]:
import pandas as pd

weekly_counts = (
    df
    .groupby([pd.Grouper(key='created_at', freq='W'), 'label'])
    .size()
    .reset_index(name='count')
)

weekly_pivot = (
    weekly_counts
    .pivot(index='created_at', columns='label', values='count')
    .fillna(0)
    .reset_index()
)



In [5]:
import plotly.io as pio

pio.renderers.default = 'notebook_connected'


In [6]:
weekly_three = weekly_pivot.copy()

weekly_three['positive'] = (
    weekly_three['Very Positive'] +
    weekly_three['Positive']
)

weekly_three['negative'] = (
    weekly_three['Very Negative'] +
    weekly_three['Negative']
)

weekly_three['neutral'] = weekly_three['Neutral']

weekly_three = weekly_three[['created_at', 'positive', 'neutral', 'negative']]


In [7]:
import plotly.express as px

sentiment_cols = ['positive', 'neutral', 'negative']
cbf = ['#0072B2','#E69F00','#D35FB7']

fig = px.area(
    weekly_three,
    x='created_at',
    y=sentiment_cols,
    title='Weekly Tweet Sentiment Over Time',
    labels={'value':'Tweet Count','created_at':'Week'},
    template='plotly_white',
    color_discrete_sequence=cbf
)

fig.update_xaxes(dtick="M1", tickformat="%b %Y", tickangle=-45)
fig.update_layout(xaxis_title='Week', yaxis_title='Number of Tweets', legend_title='Sentiment')
fig.show()


In [10]:
import pandas as pd
import ast
import psycopg2

conn = psycopg2.connect(
    dbname="DataChallenge",
    user="dbadmin",
    password="BZ6uHRGxki6a7qD",
    host="dcpostgres.postgres.database.azure.com",
    port="5432",
    sslmode="require"
)

query = """
SELECT
  created_at,
  senti_raw_tabularis,
  text
FROM tweets
WHERE
  senti_raw_tabularis IS NOT NULL
  AND created_at            IS NOT NULL;
"""

df_full = pd.read_sql_query(query, conn)

df_full['created_at'] = pd.to_datetime(df_full['created_at'])
df_full['label']      = df_full['senti_raw_tabularis'].apply(lambda x: ast.literal_eval(x)['label'])



Loaded rows: 6094135


,created_at,senti_raw_tabularis,text,label
0,2019-06-12 21:04:49,"{'label': 'Very Negative', 'score': 0.43800345...",@Ryanair why am I getting an email telling me ...,Very Negative
1,2019-06-12 21:06:20,"{'label': 'Very Positive', 'score': 0.64106345...",Top news today: @BLQairport: 'Buona serata dal...,Very Positive
2,2019-06-12 21:06:43,"{'label': 'Very Negative', 'score': 0.58267146...","@AmericanAir HEY, why charge for premium seats...",Very Negative
3,2019-06-12 21:10:06,"{'label': 'Very Positive', 'score': 0.51672941...",RT @BillOReilly: Way to go @Delta. I will rem...,Very Positive
4,2019-06-12 21:25:51,"{'label': 'Neutral', 'score': 0.5808106660842896}","RT @SanguDelle: 3 white guys ahead of me, no q...",Neutral


In [11]:
df_klm = df_full[df_full['text'].str.contains('@KLM', case=False, na=False)].copy()


Total @KLM‐mention tweets: 235871


,created_at,label,text
101,2019-06-13 09:59:42,Very Negative,"RT @KLM: Honk honk, meep meep, tuut tuut! 📢 Do..."
122,2019-06-13 11:11:07,Very Negative,"RT @daniyalyos: So, @EtihadAirways @EtihadHelp..."
145,2019-06-13 12:46:06,Very Negative,.@KLM_UK Your online costumer service is terri...
200,2019-06-13 17:55:28,Very Positive,@WinnyfredDenise @KLM @Schiphol De piloot heef...
210,2019-06-13 18:45:40,Negative,@KLM so what’s the issue this week with 21:05 ...


In [12]:
import pandas as pd

weekly_klm = (
    df_klm
    .groupby([pd.Grouper(key='created_at', freq='W'), 'label'])
    .size()
    .reset_index(name='count')
)

weekly_klm_pivot = (
    weekly_klm
    .pivot(index='created_at', columns='label', values='count')
    .fillna(0)
    .reset_index()
)


label created_at  Negative  Neutral  Positive  Very Negative  Very Positive
0     2019-05-26        90      644        81            316            435
1     2019-06-02       230     1272       158            645           1174
2     2019-06-09       268     1546       199            840           1544
3     2019-06-16       283     1218       169            758           1982
4     2019-06-23       280     1584       189            737           1509

Sentiment columns: ['Negative', 'Neutral', 'Positive', 'Very Negative', 'Very Positive']


In [15]:
import plotly.express as px

klm_sentiments = [c for c in weekly_klm_pivot.columns if c != 'created_at']

weekly_three_klm = weekly_klm_pivot.copy()

weekly_three_klm['positive'] = (
    weekly_three_klm['Very Positive'] +
    weekly_three_klm['Positive']
)

weekly_three_klm['negative'] = (
    weekly_three_klm['Very Negative'] +
    weekly_three_klm['Negative']
)

weekly_three_klm['neutral'] = weekly_three_klm['Neutral']

weekly_three_klm = weekly_three_klm[['created_at', 'positive', 'neutral', 'negative']]



In [17]:
import plotly.express as px

sentiment_cols_klm = ['positive', 'neutral', 'negative']
cbf = ['#0072B2','#E69F00','#D35FB7']

fig = px.area(
    weekly_three_klm,
    x='created_at',
    y=sentiment_cols,
    title='Weekly Tweet Sentiment Over Time for KLM',
    labels={'value':'Tweet Count','created_at':'Week'},
    template='plotly_white',
    color_discrete_sequence=cbf
)

fig.update_xaxes(dtick="M1", tickformat="%b %Y", tickangle=-45)
fig.update_layout(xaxis_title='Week', yaxis_title='Number of Tweets', legend_title='Sentiment')
fig.show()